In [ ]:
# # Set up Kaggle API key
import os
# Assuming you've uploaded your kaggle.json file
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

# # Download and unzip the "img-net-10percent" dataset
!kaggle datasets download -d kerrit/imagenet1kmediumtest-10k
!unzip imagenet1kmediumtest-10k


In [19]:
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
tf.config.run_functions_eagerly(True)
# Define paths and directories
data_dir = "/content/test_10000/test_10000"  # Path to the image folder

# List subdirectories in the dataset directory
subdirectories = os.listdir(data_dir)

# Load and preprocess the grayscale images
images = []
image_shape = (256,256)

In [21]:

for subdirectory in subdirectories:
    subdirectory_path = os.path.join(data_dir, subdirectory)
    image_files = os.listdir(subdirectory_path)

    for image_file in image_files:
        image_path = os.path.join(subdirectory_path, image_file)
        image = Image.open(image_path).convert('L')  # Load and convert to grayscale
        image = image.resize(image_shape)  # Resize to the specified dimensions
        image = np.array(image)

        images.append(image)

images = np.stack(images)  # Convert the list of images to a NumPy array
images = images.reshape(images.shape[0], *image_shape, 1)

# Create an ImageDataGenerator
datagen = ImageDataGenerator(rescale=1.0/255)  # You can add more augmentation settings

# Create a data generator
batch_size = 16
data_generator = datagen.flow(images, images, batch_size=batch_size)

# Create the model (same as before)
def create_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    return model

model = create_model()
model.compile(optimizer='adam', loss='mse')  # Mean squared error loss for regression task

# Train the model using the data generator
steps_per_epoch = len(images) // batch_size



In [ ]:
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse')  # Mean squared error loss for regression task

# Train the model using the data generator
steps_per_epoch = len(images) // batch_size
model.fit(data_generator, steps_per_epoch=steps_per_epoch, epochs=10)

Epoch 1/10
